In [ ]:
# importing required libraries, modules and pretrained model 

import os
import tensorflow as tf
from tensorflow import keras
from skimage import io
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPool2D, Dropout, Flatten, AveragePooling2D
from tensorflow.keras.optimizers import RMSprop, SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.applications.xception import Xception


SEED = 42
SIZE = (224, 224)
BATCH_SIZE = 32

pd.set_option('display.max_rows', None)

In [ ]:
# training and validation data

labels = pd.read_csv('../input/dog-breed-identification/labels.csv')
labels.head()

In [ ]:
labels['id'] = labels['id'].apply(lambda x: x + '.jpg')
labels.head()

In [ ]:
labels['breed'].value_counts().plot.bar(figsize=(16, 8))

****The data distribution is kind of ok. Maybe some data augmentaion to increase the lower quantity classes can be useful. Hopefully this part will be added to the notebook soon.****

In [ ]:
# ImageDatagenerator to load the images in batches and perform data augmentation

data_generator = ImageDataGenerator(rescale= 1./255, validation_split=0.2, rotation_range=20,
                                    zoom_range=0.1, width_shift_range=0.2, height_shift_range=0.2,
                                    shear_range=0.1, horizontal_flip=True, fill_mode="nearest")

In [ ]:
train_generator = data_generator.flow_from_dataframe(labels, directory='../input/dog-breed-identification/train/', x_col='id', y_col='breed', target_size=SIZE, class_mode='categorical', batch_size=BATCH_SIZE, shuffle=True, seed=SEED, subset='training')
val_generator = data_generator.flow_from_dataframe(labels, directory='../input/dog-breed-identification/train/', x_col='id', y_col='breed', target_size=SIZE, class_mode='categorical', batch_size=BATCH_SIZE, shuffle=True, seed=SEED, subset='validation')

In [ ]:
# 12 images after augmentation

img, label = next(train_generator)

fig = plt.figure(figsize=(15, 10))

for i in range(12):
    fig.add_subplot(3, 4, i+1)
    plt.imshow(img[i])
    plt.axis('off')

In [ ]:
# callbacks that will be used during training

early_stopping = EarlyStopping(monitor='val_loss', mode = 'min', patience=10)
checkpoint = ModelCheckpoint(filepath = './weights.hdf5', verbose=1, save_best_only=True)\

****The pretrained Xeption model on the imagenet dataset will be used. The Inception and ResNet models were also tried but have not performed better.****

In [ ]:
base_model = Xception(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))

base_model.summary()

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
head_model = AveragePooling2D(pool_size=(4, 4))(base_model.output)
head_model = Flatten(name='flatten')(head_model)
head_model = Dense(1024, activation='relu')(head_model)
head_model = Dropout(0.3)(head_model)
head_model = Dense(512, activation='relu')(head_model)
head_model = Dropout(0.3)(head_model)
head_model = Dense(120, activation='softmax')(head_model)

In [ ]:
model = Model(inputs=base_model.input, outputs=head_model)
optimizer = SGD(learning_rate=0.1, momentum=0.9, decay=0.01)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

**The model will be trained in 3 cycles. The first cycle only the added classifier's layers to the network "head_model" will be trainable with a relativly high learing rate. During the second cycle half of the xception model's layers will be unfrozen while the third cycle all of the layers will be unfrozen. During the second and third cycles the learning rate of the optimizer will be reduced.**

In [ ]:
#first cycle

history1 = model.fit(train_generator, epochs=5, validation_data=val_generator, callbacks=[checkpoint])

In [ ]:
plt.plot(history1.history['accuracy'], label='training accuracy')
plt.plot(history1.history['val_accuracy'], label='validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Training and Validation Accuracy')
plt.legend(loc='lower right')

In [ ]:
plt.plot(history1.history['loss'], label='training loss')
plt.plot(history1.history['val_loss'], label='validation loss')
plt.xlabel('Epochs')
plt.ylabel('Training and Validation Loss')
plt.legend()

In [ ]:
#second cycle

for layer in base_model.layers[len(base_model.layers)//2:]:
    layer.trainable = True

optimizer = SGD(learning_rate=0.01, momentum=0.9, decay=0.001)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

In [ ]:
history2 = model.fit(train_generator, epochs=10, validation_data=val_generator, callbacks=[checkpoint])

In [ ]:
plt.plot(history2.history['accuracy'], label='training accuracy')
plt.plot(history2.history['val_accuracy'], label='validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Training and Validation Accuracy')
plt.legend(loc='lower right')

In [ ]:
plt.plot(history2.history['loss'], label='training loss')
plt.plot(history2.history['val_loss'], label='validation loss')
plt.xlabel('Epochs')
plt.ylabel('Training and Validation Loss')
plt.legend()

**The model clearly started overfitting after after couple of epochs so the best weights saved in the weigh.hdf5 file will be loaded before the third cycle.**

In [ ]:
model.load_weights('./weights.hdf5')

In [ ]:
# third cycle

for layer in base_model.layers:
    layer.trainable = True

optimizer = SGD(learning_rate=0.01, momentum=0.9, decay=0.001)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

In [ ]:
history3 = model.fit(train_generator, epochs=100, validation_data=val_generator, callbacks=[checkpoint, early_stopping])

In [ ]:
plt.plot(history3.history['accuracy'], label='training accuracy')
plt.plot(history3.history['val_accuracy'], label='validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Training and Validation Accuracy')
plt.legend(loc='lower right')

In [ ]:
plt.plot(history3.history['loss'], label='training loss')
plt.plot(history3.history['val_loss'], label='validation loss')
plt.xlabel('Epochs')
plt.ylabel('Training and Validation Loss')
plt.legend()

**The model performance did not improve at all so we will return to the best weights reached in the second cycle.**

In [ ]:
# loading the testset

test_images_files_names = os.listdir('../input/dog-breed-identification/test/')
test_set = pd.DataFrame(test_images_files_names, columns=['id'])
test_set.head()

In [ ]:
test_data_generator = ImageDataGenerator(rescale= 1./255)
test_generator = test_data_generator.flow_from_dataframe(test_set, directory='../input/dog-breed-identification/test/', x_col='id',target_size=SIZE, class_mode=None, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
model.load_weights('./weights.hdf5')

In [ ]:
y_prop = model.predict(test_generator)

In [ ]:
results = pd.DataFrame(columns=["id"] + [*train_generator.class_indices.keys()])
results

In [ ]:
results["id"] = [os.path.splitext(file)[0] for file in os.listdir('../input/dog-breed-identification/test/')]
results.head()

In [ ]:
results[[*train_generator.class_indices.keys()]] = y_prop
results.head()

results.to_csv("results.csv",index=False)